In [1]:
import sys, os
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras

from tensorflow.keras.utils import img_to_array
from keras import layers
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.applications.vgg16 import VGG16

In [2]:
train_dir = "/Users/taikisakai/git_rep/TransferLearning-CNN-/dataset/train"
test_dir = "/Users/taikisakai/git_rep/TransferLearning-CNN-/dataset/test"
img_size = 255
num_classes = 3
class_names = ["Crazing", "Inclusion", "Patches"]

In [3]:
augment = keras.Sequential(
    [layers.RandomFlip("horizontal_and_vertical"), 
     layers.RandomRotation(0.2)]
)

2023-11-21 22:59:55.992528: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2023-11-21 22:59:55.992545: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-11-21 22:59:55.992549: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-11-21 22:59:55.992571: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-21 22:59:55.992583: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
data_dir = "/Users/taikisakai/git_rep/TransferLearning-CNN-/dataset/"
INPUT_TFRECORD_TRAIN = os.path.join(data_dir, "train_tfrecords")
INPUT_TFRECORD_TEST = os.path.join(data_dir, "test_tfrecords")

In [5]:
class LoadTfrecords:

    def __init__(self, BATCH_SIZE):
        self.BATCH_SIZE = BATCH_SIZE


    def parse_train(self, serialized):
        features = {"image": tf.io.FixedLenFeature([], tf.string), 
                    "label": tf.io.FixedLenFeature([], tf.string)}
        
        parsed = tf.io.parse_single_example(serialized=serialized, 
                                            features=features)
        
        image_raw = parsed["image"]
        label_raw = parsed["label"]

        images = tf.io.decode_raw(image_raw, tf.uint8)
        images = tf.cast(images, tf.float32) / 255
        images = tf.reshape(images, [200, 200, 3])

        labels = tf.io.decode_raw(label_raw, tf.float32)
        labels = tf.reshape(labels, [3])

        return images, labels
    

    def parse_test(self, serialized):
        features = {"image": tf.io.FixedLenFeature([], tf.string), 
                    "label": tf.io.FixedLenFeature([], tf.string)}
        
        parsed = tf.io.parse_single_example(serialized=serialized, 
                                            features=features)
        
        image_raw = parsed["image"]
        label_raw = parsed["label"]

        images = tf.io.decode_raw(image_raw, tf.uint8)
        images = tf.cast(images, tf.float32) / 255
        images = tf.reshape(images, [200, 200, 3])

        labels = tf.io.decode_raw(label_raw, tf.float32)
        labels = tf.reshape(labels, [3])

        return images, labels


In [6]:
parse_data = LoadTfrecords(BATCH_SIZE=32)

trainset = tf.data.TFRecordDataset(INPUT_TFRECORD_TRAIN)
trainset = trainset.map(parse_data.parse_train)
trainset = trainset.shuffle(buffer_size=828)
trainset = trainset.repeat(-1)
trainset = trainset.batch(32)
trainset = trainset.prefetch(buffer_size=4)

testset = tf.data.TFRecordDataset(INPUT_TFRECORD_TEST)
testset = testset.map(parse_data.parse_test)
testset = testset.batch(32)
testset = testset.prefetch(buffer_size=2)

In [7]:
conv_base = keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))
#conv_base.trainable = False

In [8]:
model = keras.models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(3, activation='softmax'))


In [9]:
inputs = Input(shape=(200, 200, 3))
x = augment(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation='softmax')(x)
model = keras.Model(inputs, outputs)
model.compile(loss='categorical_crossentropy', 
              optimizer=keras.optimizers.SGD(learning_rate=1e-5), 
              metrics=['acc'])


In [10]:
model.compile(loss='categorical_crossentropy', 
              optimizer=keras.optimizers.SGD(learning_rate=1e-5), 
              metrics=['acc'])

In [12]:
history = model.fit(trainset, 
                    steps_per_epoch=20, 
                    epochs=10, 
                    validation_data=testset, 
                    verbose=1)

Epoch 1/10


2023-11-21 23:00:29.277766: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13396327832007287016
2023-11-21 23:00:29.277790: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5677204148285014177
2023-11-21 23:00:29.277797: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4776939947513444333
2023-11-21 23:00:29.277805: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 5371450645822246281
2023-11-21 23:00:29.277812: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2794951171050062619
2023-11-21 23:00:29.277818: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 242914742123398161
2023-11-21 23:00:29.277825: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv ite

InvalidArgumentError: Graph execution error:

Detected at node Reshape defined at (most recent call last):
<stack traces unavailable>
Detected at node Reshape defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) INVALID_ARGUMENT:  Input to reshape is a tensor with 40000 values, but the requested shape has 120000
	 [[{{node Reshape}}]]
	 [[IteratorGetNext]]
	 [[model/sequential/random_flip/stateful_uniform_full_int_1/Bitcast_1/_24]]
  (1) INVALID_ARGUMENT:  Input to reshape is a tensor with 40000 values, but the requested shape has 120000
	 [[{{node Reshape}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_3435]